In [34]:
from dask.distributed import Client, progress, get_worker
client = Client("tcp://131.180.106.138:8786")

In [35]:
file_list = [
    'http://www.nwhiker.com/wallpaper/wp01.jpg',
    'http://www.nwhiker.com/wallpaper/wp02.jpg',
    'http://www.nwhiker.com/wallpaper/wp03.jpg',
    'http://www.nwhiker.com/wallpaper/wp04.jpg',
    'http://www.nwhiker.com/wallpaper/wp05.jpg',
    'http://www.nwhiker.com/wallpaper/wp06.jpg',
    'http://www.nwhiker.com/wallpaper/wp07.jpg',
    'http://www.nwhiker.com/wallpaper/wp08.jpg',
    'http://www.nwhiker.com/wallpaper/wp09.jpg',
    'http://www.nwhiker.com/wallpaper/wp10.jpg'
]

In [36]:

def run_in_worker(file_name):
    import requests
    import os
    from xfZlibWrapper import xfZlibWrapper

    # Download to filesystem
    print(f"Downloading file from URL {file_name}")
    os.makedirs(os.path.dirname("data/"), exist_ok=True)    
    file_path = 'data/' + file_name.split("/")[-1]
    r = requests.get(file_name)
    open(file_path, 'wb').write(r.content)
    
    
    # Call FPGA driver
    print("calling constructor")
    xfZlib = xfZlibWrapper(b"build/xclbin_xilinx_u200_xdma_201830_2_sw_emu/compress_decompress.xclbin")
    print("%%%%% done calling constructor")
    
    size = xfZlib.compress_file(file_path, file_path + ".gz")
    print(f'Compressed from {os.path.getsize(file_path)} to {size} bytes')
    return size


In [37]:
# distributed_data = client.scatter(file_list)
# print(len(distributed_data))
futures = client.map(run_in_worker,  file_list)